In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train_df = pd.read_csv('./data/trainingset.csv')
test_df = pd.read_csv('./data/testset.csv')
train_df.head()

C:\Users\justi\Anaconda3\envs\deeplearning\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (50,172,255,256,257,258,268,280,376) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\justi\Anaconda3\envs\deeplearning\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (50,255,256,257,258,260,268) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cntryid,cntryid_e,age_r,gender_r,computerexperience,nativespeaker,edlevel3,monthlyincpr,yearlyincpr,lng_home,...,v85,v50,v89,v127,v239,v224,v71,v105,row,uni
0,Russian Federation,Russian Federation,42.0,Female,Yes,Yes,High,25 to less than 50,25 to less than 50,999,...,Agree,Agree,NaN,NaN,233.0,9996.0,8521,9996.0,95754,jw3580
1,United States,United States,NaN,Male,Yes,Yes,High,75 to less than 90,75 to less than 90,999,...,Agree,Neither agree nor disagree,NaN,NaN,9999.0,9996.0,9999,9996.0,57367,jw3580
2,United States,United States,NaN,Female,Yes,Yes,High,NaN,NaN,999,...,Strongly agree,Strongly agree,NaN,NaN,9996.0,9999.0,9996,9999.0,195554,jw3580
3,Czech Republic,Czech Republic,23.0,Female,Yes,Yes,Medium,10 to less than 25,10 to less than 25,ces,...,Agree,Agree,NaN,NaN,3311.0,9996.0,6419,9996.0,42148,jw3580
4,Canada,Canada (English),NaN,Male,Yes,Yes,Medium,NaN,NaN,999,...,Disagree,Agree,NaN,NaN,9996.0,9999.0,9999,9999.0,86076,jw3580


In [3]:
# Dropping uni and v262 because the two columns have no infomration
train_df.drop(['uni', 'v262'], axis=1, inplace=True)
test_df.drop(['uni', 'v262'], axis=1, inplace=True)

# NaN processing with interpolation
train_df.fillna(train_df.mean(), inplace=True)
test_df.fillna(test_df.mean(), inplace=True)

# Pick out categorical columns
train_categ_df = train_df.select_dtypes(include=[np.object], exclude=[np.number])
train_df.drop(train_categ_df.columns, axis=1, inplace=True)
test_categ_df = test_df.select_dtypes(include=[np.object], exclude=[np.number])
test_df.drop(test_categ_df.columns, axis=1, inplace=True)

# One-hot encode
train_categ_df = pd.get_dummies(train_categ_df, dummy_na=True)
test_categ_df = pd.get_dummies(test_categ_df, dummy_na=True)
train_df = pd.concat([train_df, train_categ_df], axis=1)
test_df = pd.concat([test_df, test_categ_df], axis=1)
test_df.columns

# Divide into X and Y (for submission)
X_train_final = train_df.drop('job_performance', axis=1) # Fit on train_final's
y_train_final = train_df['job_performance']
X_test_final = test_df.drop('job_performance', axis=1) # and predict on X_test_final for submission
y_test_final = test_df['job_performance'] # Empty

# Pipeline for Model Selection and Tuning
Splitting train_df into train and test because only train_df has true labels to check accuracy

In [4]:
from sklearn.model_selection import train_test_split
df = train_df[:]
X = train_df.drop('job_performance', axis=1)
y = train_df['job_performance']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape, y_test.shape

((14000, 3329), (6000,))

In [15]:
def exceeds_benchmark(MSE):
    '''Returns True if MSE is satisfactory'''
    benchmark = 659.41
    print(f'{MSE - benchmark} to go')
    return MSE <= benchmark

# Create Baseline Model
Basic OLS regression

In [8]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_train, y_train)

0.7416984790578235

In [9]:
y_true = y_test[:]
y_pred = reg.predict(X_test)
y_pred.shape

(6000,)

In [10]:
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_true, y_pred)
MSE

89451.49947723513

In [16]:
exceeds_benchmark(MSE)

88792.08947723513 to go


False